## Modified Berggren Frost Depth
### Background

From Bianchini and Gonzalez (2012)<sup>[[6]](#6)</sup>:

> "The design of pavement structures in cold climates must account for the changes in soil properties due to the influence of freezing and thawing cycles. The calculation of frost depth is a fundamental step during the design and evaluation of pavement structures by the U.S. Department of Defense (DoD). The DoD uses the modified Berggren (ModBerg) equation to compute the frost penetration depth."
> 
The functions provided here compute the ModBerg frost depth in the same fashion as demonstrated in the The Unified Facilities Criteria (UFC) 3-130-06<sup>[[5]](#5)</sup>. Note that other tools (in particular the Pavement-Transportation Computer Assisted Structural Engineering (PCASE)) offer numerical solutions to the ModBerg equation and thus predict different values than the solutions described in the UFC and presented here. See Bianchini and Gonzalez (2012)<sup>[[6]](#6)</sup> for a discussion of those differences.

### Assumptions

- Frost depth is computed for a single layer of homogenous isotropic soil.
- Heat flow is one-dimensional with the entire soil mass at its mean annual temperature prior to the start of the freezing season. The surface temperature changes suddenly as a step function from the mean annual temperature to a temperature v<sub>s</sub> degrees below freezing and remains at this new temperature throughout the entire freezing season.
- The initial ground temperature is assumed to uniformly equal the mean annual air temperature. The upper boundary condition is represented by the surface freezing index.
- The coefficient λ considers the effect of the temperature changes in the soil mass and is a function of the fusion parameter μ and thermal ratio α. Traditionally, λ is found "manually" by using values of μ and α to consult a chart such as this one from Aldrich and Paynter (1966)<sup>[[3]](#3)</sup>:

 <img src="static/correction_coeff.png" height="400">

Here λ is found using equation _2-33_ from Aldrich (1953)<sup>[[2]](#2)</sup> implemented like so:

```python
def compute_lambda_coeff(mu, thermal_ratio):
    """Compute the lambda coeffcient (dimensionless).
    Args:
        mu: the fusion parameter (dimensionless)
        thermal_ratio: thermal ratio (dimensionless)
    Returns
        lc: the lambda coeffcient value (dimensionless).
    """
    lc = 1.0 / (np.sqrt(1 + (mu * (thermal_ratio + 0.5))))
    return round(lc, 2)
```

This method removes manual consultation of the above figure and should produce coefficients suitable for high latitudes (where the thermal ratio is low), though may overestimate frost depths for more temperate climates.

### User Inputs

- Soil Factors
  - Thermal Conductivity (BTU/hr • ft • °F)
  - Dry Density (lbs per cubic foot)
  - Gravimetric Water Content (%)
- Climate Factors
  - Climate Model for Projected Mean Annual Temperature
  - Emissions Scenario for Projected Mean Annual Temperature
  - Climate Model for Projected Air Freezing Index
  - n-factor to Convert Air Freezing Index to Surface Freezing Index
  - Duration of the Freezing Season (days)
  - Start and End Years (period over which to average mean annual temperature and freezing index projections)

#### References

<a id="1">[1]</a> W. P. Berggren, “Prediction of temperature-distribution in frozen soils,” Trans. AGU, vol. 24, no. 3, p. 71, 1943, doi: 10.1029/TR024i003p00071.

<a id="2">[2]</a> H. P. Aldrich and H. M. Paynter, “Analytical Studies of Freezing and Thawing of Soils,” Arctic Construction and Frost Effects Laboratory, Corps of Engineers, U.S. Army, Boston, MA, First Interim Technical Report 42, Jun. 1953.

<a id="3">[3]</a> H. P. Aldrich and H. M. Paynter, “Depth of Frost Penetration in Non-Uniform Soil,” Conducted for CORPS OF ENGINEERS, U. S. ARMY by U.S. ARMY MATERIEL COMMAND COLD REGIONS RESEARCH & ENGINEERING LABORATORY, Hanover, New Hampshire, Special Report 104, Oct. 1966.

<a id="4">[4]</a> “ARCTIC AND SUBARCTIC CONSTRUCTION CALCULATION METHODS FOR DETERMINATION OF DEPTHS OF FREEZE AND THAW IN SOILS,” Joint Departments of the Army and Air Force, USA, Technical Manual Tm 5-852-6 / AFR 88-19, Volume 6, Jan. 1988.

<a id="5">[5]</a> “Calculation Methods for Determination of Depth of Freeze and Thaw in Soil: Arctic and Subarctic Construction,” U.S. Army Corps of Engineers, United Facilities Criteria UFC 3-130-06, Jan. 2004. [Online]. Available: https://www.wbdg.org/FFC/DOD/UFC/INACTIVE/ufc_3_130_06_2004.pdf

<a id="6">[6]</a> A. Bianchini and C. R. Gonzalez, “Pavement-Transportation Computer Assisted Structural Engineering (PCASE) Implementation of the Modified Berggren (ModBerg) Equation for Computing the Frost Penetration Depth within Pavement Structures,” Geotechnical and Structures Laboratory U.S. Army Engineer Research and Development Center, Vicksburg, MS, Final ERDC/GSL TR-12-15, Apr. 2012. doi: 10.21236/ADA559915.

<a id="7">[7]</a> H. P. Aldrich, “Frost Penetration Below Highway And Airfield Pavements,” p. 26.


In [ ]:
from modberg import compute_modified_berggren

In [ ]:
import micropip
await micropip.install("ipywidgets")
import ipywidgets as widgets
from ipywidgets import interact, Layout, Output
from IPython.display import display

In [ ]:
# climate futures for freezing index projections (all RCP 8.5)
fi_models = ["GFDL-CM3", "NCAR-CCSM4"]

# climate futures for mean annual temperature projections
mat_models = ["GFDL-CM3", "NCAR-CCSM4", "MRI-CGCM3", "GISS-E2-R", "IPSL-CM5A-LR"]
mat_scenarios = ["rcp45", "rcp60", "rcp85"]

In [ ]:
# location inputs
lat_box = widgets.BoundedFloatText(
    value=65.0,
    min=51.3,
    max=71.3,
    step=0.1,
    description="Latitude",
    continuous_update=False,
)
lon_box = widgets.BoundedFloatText(
    value=-147.0,
    min=-179.1,
    max=-130.0,
    step=0.1,
    description="Longitude",
    continuous_update=False,
)

# climate inputs
d_slider = widgets.IntSlider(
    value=200,
    min=1,
    max=365,
    step=1,
    description="Freezing Duration (days):",
    layout=Layout(width="500px"),
    style={"description_width": "initial"},
)
n_slider = widgets.FloatSlider(
    value=0.75,
    min=0.10,
    max=1.00,
    step=0.1,
    description="Surface n-factor for Freezing Index:",
    layout=Layout(width="500px"),
    style={"description_width": "initial"},
)
start_slider = widgets.IntSlider(
    value=2030,
    min=2007,
    max=2100,
    description="Start Year:",
    layout=Layout(width="500px"),
)
stop_slider = widgets.IntSlider(
    value=2040,
    min=2007,
    max=2100,
    description="End Year:",
    layout=Layout(width="500px"),
)

# soil property inputs
dry_ro_slider = widgets.IntSlider(
    value=100,
    min=20,
    max=135,
    step=1,
    description="Soil Dry Density (lbs per cubic foot):",
    layout=Layout(width="500px"),
    style={"description_width": "initial"},
)
wc_pct_slider = widgets.IntSlider(
    value=15,
    min=1,
    max=50,
    step=1,
    description="Soil Gravimetric Water Content (%):",
    layout=Layout(width="500px"),
    style={"description_width": "initial"},
)
k_avg_slider = widgets.FloatSlider(
    value=0.78,
    min=0.1,
    max=2,
    step=0.1,
    description="Soil Thermal Conductivity (BTU/hr • ft • °F):",
    layout=Layout(width="500px"),
    style={"description_width": "initial"},
)

# climate future dropdowns
fi_model_dropdown = widgets.Dropdown(
    options=fi_models,
    description="Freezing Index Model (RCP 8.5 scenario only):",
    layout=Layout(width="500px"),
    style={"description_width": "initial"},
)
mat_model_dropdown = widgets.Dropdown(
    options=mat_models,
    description="Mean Annual Temperature Model:",
    layout=Layout(width="500px"),
    style={"description_width": "initial"},
)
mat_scenario_dropdown = widgets.Dropdown(
    options=mat_scenarios,
    description="Mean Annual Temperature Scenario:",
    layout=Layout(width="500px"),
    style={"description_width": "initial"},
)

# create output widget for displaying the result
output = Output()


# function to compute and display the result
def compute_and_display():
    with output:
        output.clear_output()
        result = compute_modified_berggren(
            dry_ro=dry_ro_slider.value,
            wc_pct=wc_pct_slider.value,
            d=d_slider.value,
            n=n_slider.value,
            k_avg=k_avg_slider.value,
            lat=lat_box.value,
            lon=lon_box.value,
            model=mat_model_dropdown.value,
            fi_model=fi_model_dropdown.value,
            scenario=mat_scenario_dropdown.value,
            year_start=start_slider.value,
            year_end=stop_slider.value,
        )
        print(f"Frost Depth: {result} feet")


# create button to trigger the computation
compute_button = widgets.Button(description="Compute Modified Berggren Frost Depth")
compute_button.on_click(lambda btn: compute_and_display())

# create container to organize the widgets
widget_container = widgets.VBox(
    [
        lat_box,
        lon_box,
        d_slider,
        n_slider,
        start_slider,
        stop_slider,
        dry_ro_slider,
        wc_pct_slider,
        k_avg_slider,
        fi_model_dropdown,
        mat_model_dropdown,
        mat_scenario_dropdown,
        compute_button,
        output,
    ]
)

widget_container
